In [2]:
from FileUtils import FileUtils
import matplotlib.pyplot as plt 
import os
import torch
from Settings import Settings
from ReplayMemory import Transition, ReplayMemory

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
data = FileUtils(100)
data.filter_by_class("cat")

Start to load data
Start to load images: 100!
Start to load annotations: 100!
End


In [4]:
from Agent import Agent

In [7]:
agent = Agent(data.images[0])
print(agent.history)
agent.hierarchical_move(5)
agent.update_history_vector(5)
print(agent.history)
print(agent.history)
agent.hierarchical_move(1)
agent.update_history_vector(1)
print(agent.history)
print(agent.history)
agent.hierarchical_move(3)
agent.update_history_vector(3)
print(agent.history)
print(agent.history)
agent.hierarchical_move(2)
agent.update_history_vector(2)
print(agent.history)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1., 0.],
       

In [8]:
memo = ReplayMemory(10)

In [9]:
from Models import QModel

In [10]:
q_model = QModel()

In [12]:
for i in range(20):
    memo.push(agent.get_state(), agent.get_next_action(q_model, 0.5), agent.get_state(), torch.tensor(1))
    memo.push(agent.get_state(), agent.get_next_action(q_model, 0.5), None, torch.tensor(3))

In [13]:
transitions = memo.sample(5)

In [14]:
transitions

[Transition(state=tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), action=tensor(4), next_state=None, reward=tensor(3)),
 Transition(state=tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), action=tensor(6), next_state=tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), reward=tensor(1)),
 Transition(state=tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), action=tensor(1), next_state=None, reward=tensor(3)),
 Transition(state=tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), action=tensor(4), next_state=tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), reward=tensor(1)),
 Transition(state=tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), action=tensor(6), next_state=None, reward=tensor(3))]

In [15]:
batch = Transition(*zip(*transitions))

In [16]:
print(batch)

Transition(state=(tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.]])), action=(tensor(4), tensor(6), tensor(1), tensor(4), tensor(6)), next_state=(None, tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), None, tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), None), reward=(tensor(3), tensor(1), tensor(3), tensor(1), tensor(3)))


In [17]:
def use_cuda(tensor, cuda=Settings.cuda):
    if cuda:
        tensor = tensor.cuda()
    return tensor


In [18]:
non_finals = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool)
non_finals = use_cuda(non_finals)
non_finals

tensor([False,  True, False,  True, False])

In [19]:

next_states = [s for s in batch.next_state if s is not None]
non_final_next_states = torch.cat(next_states).type(torch.float)
non_final_next_states = use_cuda(non_final_next_states)

non_final_next_states.shape

torch.Size([2, 25112])

In [20]:
state_batch = torch.cat(batch.state).type(torch.float)
state_batch = use_cuda(state_batch)
state_batch.shape

torch.Size([5, 25112])

In [21]:
from torch.autograd import Variable

In [22]:
action_batch = torch.tensor(batch.action, dtype=torch.long).view(-1, 1)
action_batch = use_cuda(action_batch)

In [23]:
action_batch

tensor([[4],
        [6],
        [1],
        [4],
        [6]])

In [24]:
reward_batch = torch.tensor(batch.reward, dtype=torch.float).view(-1, 1)
reward_batch = use_cuda(reward_batch)

In [25]:
reward_batch

tensor([[3.],
        [1.],
        [3.],
        [1.],
        [3.]])

In [29]:
 a = q_model(state_batch)

In [30]:
a

tensor([[0.1058, 0.2052, 0.0754, 0.1215, 0.1793, 0.3128],
        [0.3297, 0.0954, 0.0944, 0.0901, 0.1247, 0.2657],
        [0.1678, 0.0741, 0.1131, 0.0831, 0.1994, 0.3625],
        [0.1281, 0.1320, 0.1753, 0.0259, 0.2544, 0.2842],
        [0.0298, 0.3376, 0.1757, 0.2602, 0.1122, 0.0844]],
       grad_fn=<SoftmaxBackward>)

In [31]:
state_action_values = a.gather(1, action_batch - 1) 

In [32]:
state_action_values

tensor([[0.1215],
        [0.2657],
        [0.1678],
        [0.0259],
        [0.0844]], grad_fn=<GatherBackward>)

In [34]:
a.max(1)[0].view(-1, 1)

tensor([[0.3128],
        [0.3297],
        [0.3625],
        [0.2842],
        [0.3376]], grad_fn=<ViewBackward>)

In [35]:
action_batch

tensor([[4],
        [6],
        [1],
        [4],
        [6]])

In [36]:
next_state_values = torch.zeros(5)
next_state_values = use_cuda(next_state_values)

next_state_values[non_finals] = q_model(non_final_next_states).max(1)[0].detach()
next_state_values.view(-1, 1)

tensor([[0.0000],
        [0.3765],
        [0.0000],
        [0.4212],
        [0.0000]])

In [37]:
next_state_values = next_state_values.view(-1, 1)

In [38]:
expected_state_action_values = (next_state_values * Settings.gamma) + reward_batch


In [39]:
expected_state_action_values

tensor([[3.0000],
        [1.3389],
        [3.0000],
        [1.3791],
        [3.0000]])

In [40]:
state_action_values.shape

torch.Size([5, 1])

In [41]:
import torch.nn.functional as F

In [42]:
loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)

In [43]:
loss

tensor(1.7105, grad_fn=<SmoothL1LossBackward>)